In [2]:
import pycountry

def get_iso_code(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

In [ ]:
# VBA code to loop over all Excel files in a folder and subfolders and replace "Eur_Dev" with "Eur_West"
# To use: Open Excel, press ALT+F11, Insert > Module, and paste this code. Run with F5.

Sub ReplaceEurDevInAllFiles()
    Dim fso As Object
    Dim folderPath As String
    Dim fileList As Collection
    Dim filePath As Variant
    
    ' Prompt user to select folder
    With Application.FileDialog(msoFileDialogFolderPicker)
        .Title = "Select folder containing Excel files"
        If .Show = -1 Then
            folderPath = .SelectedItems(1)
        Else
            MsgBox "No folder selected. Exiting."
            Exit Sub
        End If
    End With
    
    Set fileList = New Collection
    Set fso = CreateObject("Scripting.FileSystemObject")
    GetAllExcelFiles fso.GetFolder(folderPath), fileList
    
    Application.ScreenUpdating = False
    Application.DisplayAlerts = False
    
    For Each filePath In fileList
        On Error Resume Next
        Dim wb As Workbook
        Set wb = Workbooks.Open(filePath, ReadOnly:=False)
        If Not wb Is Nothing Then
            Dim ws As Worksheet
            Dim cell As Range
            Dim changed As Boolean
            changed = False
            For Each ws In wb.Worksheets
                For Each cell In ws.UsedRange
                    If VarType(cell.Value) = vbString Then
                        If Trim(LCase(cell.Value)) = "eur_dev" Then
                            cell.Value = "Eur_West"
                            changed = True
                        End If
                    End If
                Next cell
            Next ws
            If changed Then wb.Save
            wb.Close SaveChanges:=False
        End If
        On Error GoTo 0
    Next filePath
    
    Application.DisplayAlerts = True
    Application.ScreenUpdating = True
    MsgBox "Replacement complete."
End Sub

' Recursive function to collect all Excel files in folder and subfolders
Sub GetAllExcelFiles(ByVal folder As Object, ByRef fileList As Collection)
    Dim file As Object
    Dim subfolder As Object
    For Each file In folder.Files
        If LCase(Right(file.Name, 5)) = ".xlsx" Or LCase(Right(file.Name, 4)) = ".xls" Then
            fileList.Add file.Path
        End If
    Next file
    For Each subfolder In folder.SubFolders
        GetAllExcelFiles subfolder, fileList
    Next subfolder
End Sub


In [ ]:
import pandas as pd

df_gem = pd.read_excel('Global-Integrated-Power-April-2025.xlsx', sheet_name='Power facilities')

df_gem['Capacity (GW)'] = df_gem['Capacity (MW)'] / 1000

df_gem['ISO_Code'] = [get_iso_code(country) for country in df_gem['Country/area']]

c:\Veda\VerveStacks\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


array(['nuclear', 'bioenergy', 'oil/gas', 'coal', 'wind', 'solar',
       'geothermal', 'hydropower'], dtype=object)

In [ ]:
df_ember = pd.read_csv('yearly_full_release_long_format.csv')
print(df_ember.head())



In [ ]:
#df_gem.groupby(['Country/area', 'Type','Status'], as_index=False)['Capacity (GW)'].sum()
# df_gem.groupby(['Status','ISO_Code'], as_index=False)['Capacity (GW)'].sum()
# pivot_df = df_gem.pivot_table(
#     index='ISO_Code',
#     columns='Status',
#     values='Capacity (GW)',
#     aggfunc='sum',
#     fill_value=0
# )
#print(pivot_df)
# Filter for solar in Germany (ISO code 'DEU')
solar_germany = df_gem[(df_gem['ISO_Code'] == 'DEU') & (df_gem['Type'].str.lower().str.contains('coal'))]

# Aggregate solar capacity in Germany by year (if 'Year' or similar column exists)
solar_by_year = solar_germany.groupby('Start year', as_index=False)['Capacity (GW)'].sum()
print(solar_by_year)

# Optionally, show the first few rows of solar facilities in Germany
#print("\nSample of solar facilities in Germany:")
#print(solar_germany.head())
#df_gem['Type'].unique()


#df_gem.columns

df_ember.columns

#df_ember['Subcategory'].unique()


In [ ]:
# Compare total operating coal capacity in Germany between GEM and Ember

# GEM: Filter for Germany, coal, and operating status
gem_coal_operating = df_gem[
    (df_gem['ISO_Code'] == 'DEU') &
    (df_gem['Type'].str.lower().str.contains('coal')) &
    (df_gem['Status'].str.lower() == 'operating')
]

total_gem_coal_capacity_gw = gem_coal_operating['Capacity (GW)'].sum()
print(f"Total operating coal capacity in Germany (GEM): {total_gem_coal_capacity_gw:.2f} GW")

# EMBER: Try to find the relevant columns for country and fuel type
ember_coal_operating = df_ember[
    (df_ember['Country code'] == 'DEU') &
    (df_ember['Category'] == 'Capacity') &
    (df_ember['Subcategory'] == 'Fuel') &
    (df_ember['Variable'] == 'Coal')
]


total_ember_coal_capacity_gw = ember_coal_operating.groupby('Year', as_index=False)['Value'].sum()
print(total_ember_coal_capacity_gw) 

import matplotlib.pyplot as plt

# Plot Ember coal capacity as a column chart (bar chart) by year
plt.figure(figsize=(8, 5))
plt.bar(total_ember_coal_capacity_gw['Year'], total_ember_coal_capacity_gw['Value'], color='steelblue')
plt.xlabel('Year')
plt.ylabel('Coal Capacity (GW)')
plt.title('Total Operating Coal Capacity in Germany (Ember)')
plt.tight_layout()
plt.show()
